<a href="https://colab.research.google.com/github/lukuenya/Bispectrum_Analysis/blob/master/Bispectrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import cupy as cp
# import scipy.io.wavfile as wavfile
# import matplotlib.pyplot as plt
# import os

# # Parameters for bispectrum estimation
# segment_length = 1024
# overlap = segment_length // 2
# step = segment_length - overlap
# window = cp.hanning(segment_length)
# N = segment_length


# # Function to compute the bispectrum of a signal
# def compute_bispectrum(data, fs):
#     # Normalize the data
#     max_abs = cp.max(cp.abs(data))
#     if max_abs == 0:
#         return cp.zeros((N, N), dtype=cp.complex128)
#     data = data / max_abs
#     num_segments = (len(data) - overlap) // step
#     bispec_accum = cp.zeros((N, N), dtype=cp.complex128)
#     count = 0

#     for i in range(num_segments):
#         start = i * step
#         end = start + segment_length
#         segment = data[start:end]
#         if len(segment) < segment_length:
#             break
#         segment = segment * window
#         X = cp.fft.fft(segment, n=N)
#         X1 = X[:, cp.newaxis]
#         X2 = X[cp.newaxis, :]
#         indices_sum = (cp.arange(N)[:, cp.newaxis] + cp.arange(N)[cp.newaxis, :]) % N
#         X3 = cp.conj(X[indices_sum])
#         triple_product = X1 * X2 * X3
#         bispec_accum += triple_product
#         count += 1

#     if count > 0:
#         bispec = bispec_accum / count
#     else:
#         bispec = bispec_accum
#     return bispec

# # Function to visualize and optionally save the bispectrum
# def visualize_bispectrum(bispec, fs, title, save_path=None):
#     bispec_magnitude = cp.abs(bispec)
#     bispec_magnitude_db = 10 * cp.log10(bispec_magnitude + 1e-12)
#     bispec_magnitude_db_shifted = cp.fft.fftshift(bispec_magnitude_db)
#     freq_bins = cp.fft.fftfreq(N, d=1/fs)
#     freq_bins_shifted = cp.fft.fftshift(freq_bins)

#     # Convert to CPU arrays for plotting
#     bispec_magnitude_db_shifted_cpu = cp.asnumpy(bispec_magnitude_db_shifted)
#     freq_bins_shifted_cpu = cp.asnumpy(freq_bins_shifted)

#     plt.figure(figsize=(10, 8))
#     extent = [freq_bins_shifted_cpu[0], freq_bins_shifted_cpu[-1],
#               freq_bins_shifted_cpu[0], freq_bins_shifted_cpu[-1]]
#     plt.imshow(bispec_magnitude_db_shifted_cpu, extent=extent, origin='lower',
#                aspect='auto', cmap='jet')
#     plt.xlabel('Frequency f1 (Hz)')
#     plt.ylabel('Frequency f2 (Hz)')
#     plt.title(title)
#     plt.colorbar(label='Magnitude (dB)')
#     if save_path:
#         plt.savefig(save_path)
#         plt.close()
#     else:
#         plt.show()


# # Main script
# # Set the path to your dataset folder
# dataset_path = '/content/Audio_lanzhou_2015_org'  # Update this path

# # Directory to save plots
# output_dir = '/content/results'  # Update this path if needed

# # Participant types and emotions
# participant_types = ['MDD', 'HC']
# emotions = ['Positive', 'Neutral', 'Negative']

# # Specify the session to process
# session = 'Interview'  # Manually set to 'Interview' or 'Read_Vocabulary'

# # Process each participant type
# for participant_type in participant_types:
#     participant_type_path = os.path.join(dataset_path, participant_type)
#     if not os.path.exists(participant_type_path):
#         continue

#     # Get list of participant directories
#     participant_dirs = [d for d in os.listdir(participant_type_path) if os.path.isdir(os.path.join(participant_type_path, d))]
#     participant_dirs.sort()  # Ensure consistent order

#     # Process each participant
#     for participant in participant_dirs:
#         participant_path = os.path.join(participant_type_path, participant)
#         print(f"Processing participant {participant} ({participant_type})...")

#         # Process the specified session
#         session_path = os.path.join(participant_path, session)
#         if not os.path.exists(session_path):
#             continue

#         # Process each emotion
#         for emotion in emotions:
#             emotion_path = os.path.join(session_path, emotion)
#             if not os.path.exists(emotion_path):
#                 continue

#             print(f"  Processing {session} - {emotion}")

#             # Initialize accumulator for bispectra
#             bispec_sums = cp.zeros((N, N), dtype=cp.complex128)
#             count = 0

#             # Get list of .wav files
#             recording_files = [f for f in os.listdir(emotion_path) if f.endswith('.wav')]
#             recording_files.sort()  # Ensure consistent order

#             # Process each recording
#             for recording_file in recording_files:
#                 file_path = os.path.join(emotion_path, recording_file)
#                 try:
#                     # Read the audio file
#                     fs, data = wavfile.read(file_path)
#                     if len(data.shape) > 1:
#                         data = data[:, 0]  # Take one channel if stereo
#                 except Exception as e:
#                     print(f"    Warning: Could not read file {file_path}. Skipping. Error: {e}")
#                     continue

#                 # Check if data is empty
#                 if len(data) == 0:
#                     print(f"    Warning: Empty audio data in file {file_path}. Skipping.")
#                     continue

#                 # Convert data to CuPy array
#                 data = cp.asarray(data, dtype=cp.float64)

#                 # Compute the bispectrum
#                 bispec = compute_bispectrum(data, fs)
#                 bispec_sums += bispec
#                 count += 1

#             # Compute average bispectrum for participant and state
#             if count > 0:
#                 avg_bispec = bispec_sums / count

#                 # Create output directory for the participant type
#                 participant_output_dir = os.path.join(output_dir, 'results', session.lower(), participant_type)
#                 os.makedirs(participant_output_dir, exist_ok=True)

#                 # Visualize average bispectrum
#                 title = f'Average Bispectrum of {participant} ({participant_type}) - {session} - {emotion}'
#                 save_filename = f'{participant}_{session}_{emotion}_average_bispectrum.png'
#                 save_path = os.path.join(participant_output_dir, save_filename)
#                 visualize_bispectrum(avg_bispec, fs, title, save_path)
#                 print(f"    Saved bispectrum plot to {save_path}")
#             else:
#                 print(f"    No valid recordings for {participant} ({participant_type}) - {session} - {emotion}.")

# print('Complete')


In [ ]:
import cupy as cp  # Use CuPy instead of NumPy for GPU acceleration
import scipy.io.wavfile as wavfile
import matplotlib.pyplot as plt
import os

# Parameters for bispectrum estimation
segment_length = 1024
overlap = segment_length // 2
step = segment_length - overlap
window = cp.hanning(segment_length)
N = segment_length

# Function to compute the bispectrum of a signal using CuPy
def compute_bispectrum(data, fs):
    # Normalize the data
    max_abs = cp.max(cp.abs(data))
    if max_abs == 0:
        return cp.zeros((N, N), dtype=cp.complex128)
    data = data / max_abs
    num_segments = (len(data) - overlap) // step
    bispec_accum = cp.zeros((N, N), dtype=cp.complex128)
    count = 0

    for i in range(num_segments):
        start = i * step
        end = start + segment_length
        segment = data[start:end]
        if len(segment) < segment_length:
            break
        segment = segment * window
        X = cp.fft.fft(segment, n=N)
        X1 = X[:, cp.newaxis]
        X2 = X[cp.newaxis, :]
        indices_sum = (cp.arange(N)[:, cp.newaxis] + cp.arange(N)[cp.newaxis, :]) % N
        X3 = cp.conj(X[indices_sum])
        triple_product = X1 * X2 * X3
        bispec_accum += triple_product
        count += 1

    if count > 0:
        bispec = bispec_accum / count
    else:
        bispec = bispec_accum
    return bispec

# Set the path to your dataset folder
dataset_path = '/kaggle/input/modma-org/audio_lanzhou_2015_org'  # Update this path

# Directory to save plots and bispectrum data
output_dir = '/kaggle/working'  # Update this path if needed

# Participant types and emotions
participant_types = ['MDD', 'HC']
emotions = ['Positive', 'Neutral', 'Negative']

# Specify the session to process
session = 'Interview'  # Manually set to 'Interview' or 'Read_Vocabulary'

# Process each participant type
for participant_type in participant_types:
    participant_type_path = os.path.join(dataset_path, participant_type)
    if not os.path.exists(participant_type_path):
        continue

    # Get list of participant directories
    participant_dirs = [d for d in os.listdir(participant_type_path) if os.path.isdir(os.path.join(participant_type_path, d))]
    participant_dirs.sort()  # Ensure consistent order

    # Process each participant
    for participant in participant_dirs:
        participant_path = os.path.join(participant_type_path, participant)
        print(f"Processing participant {participant} ({participant_type})...")

        # Process the specified session
        session_path = os.path.join(participant_path, session)
        if not os.path.exists(session_path):
            continue

        # Process each emotion
        for emotion in emotions:
            emotion_path = os.path.join(session_path, emotion)
            if not os.path.exists(emotion_path):
                continue

            print(f"  Processing {session} - {emotion}")

            # Initialize accumulator for bispectra
            bispec_sums = cp.zeros((N, N), dtype=cp.complex128)
            count = 0

            # Get list of .wav files
            recording_files = [f for f in os.listdir(emotion_path) if f.endswith('.wav')]
            recording_files.sort()  # Ensure consistent order

            # Process each recording
            for recording_file in recording_files:
                file_path = os.path.join(emotion_path, recording_file)
                try:
                    # Read the audio file
                    fs, data = wavfile.read(file_path)
                    if len(data.shape) > 1:
                        data = data[:, 0]  # Take one channel if stereo
                    data = data.astype(cp.float64)  # Ensure data is float64
                except Exception as e:
                    print(f"    Warning: Could not read file {file_path}. Skipping. Error: {e}")
                    continue

                # Check if data is empty
                if len(data) == 0:
                    print(f"    Warning: Empty audio data in file {file_path}. Skipping.")
                    continue

                # Convert data to CuPy array
                data = cp.asarray(data)

                # Compute the bispectrum
                bispec = compute_bispectrum(data, fs)
                bispec_sums += bispec
                count += 1

                # Free up memory
                del data, bispec
                cp._default_memory_pool.free_all_blocks()

            # Compute average bispectrum for participant and state
            if count > 0:
                avg_bispec = bispec_sums / count

                # Create output directory for the participant type
                participant_output_dir = os.path.join(output_dir, 'results', session.lower(), participant_type)
                os.makedirs(participant_output_dir, exist_ok=True)

                # Save the bispectrum data
                bispec_filename = f'{participant}_{session}_{emotion}_average_bispectrum.npy'
                bispec_save_path = os.path.join(participant_output_dir, bispec_filename)

                # Transfer bispectrum data to CPU for saving
                avg_bispec_cpu = cp.asnumpy(avg_bispec)
                np.save(bispec_save_path, avg_bispec_cpu)
                print(f"    Saved bispectrum data to {bispec_save_path}")

                # Free up memory
                del avg_bispec, avg_bispec_cpu
                cp._default_memory_pool.free_all_blocks()

            else:
                print(f"    No valid recordings for {participant} ({participant_type}) - {session} - {emotion}.")

print('Complete')


In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.stats import skew, kurtosis, entropy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# Function to extract features from bispectrum data
def extract_bispectrum_features(bispec, fs):
    # [Include the updated feature extraction function from previous messages]
    # Compute magnitude and phase
    bispec_magnitude = np.abs(bispec)
    bispec_phase = np.angle(bispec)

    # Flatten the bispectrum magnitude and phase
    bispec_mag_flat = bispec_magnitude.flatten()
    bispec_phase_flat = bispec_phase.flatten()

    # Avoid log(0) by adding a small epsilon
    eps = 1e-12

    # Bispectral Magnitude Features
    mean_mag = np.mean(bispec_mag_flat)
    max_mag = np.max(bispec_mag_flat)
    sum_mag = np.sum(bispec_mag_flat)
    spectral_flatness = np.exp(np.mean(np.log(bispec_mag_flat + eps))) / (mean_mag + eps)
    entropy_mag = entropy(bispec_mag_flat + eps)

    # Bispectral Phase Features
    mean_phase = np.mean(bispec_phase_flat)
    std_phase = np.std(bispec_phase_flat)
    skew_phase = skew(bispec_phase_flat)
    kurt_phase = kurtosis(bispec_phase_flat)

    # Quadratic Phase Coupling Features
    N = bispec.shape[0]  # Ensure N is defined
    indices_qpc = np.unravel_index(np.argmax(bispec_magnitude), bispec_magnitude.shape)
    qpc_strength = bispec_magnitude[indices_qpc]
    qpc_freqs = (indices_qpc[0] * fs / N, indices_qpc[1] * fs / N)

    # Frequency Band Features
    freq_bins = np.fft.fftfreq(N, d=1/fs)
    freq_bins = freq_bins[:N//2]
    bispec_magnitude_half = bispec_magnitude[:N//2, :N//2]

    bands = {
        'low': (0, 1000),
        'mid': (1000, 5000),
        'high': (5000, fs/2)
    }

    band_energies = {}
    for band_name, (fmin, fmax) in bands.items():
        idx = np.where((freq_bins >= fmin) & (freq_bins < fmax))[0]
        band_energy = np.sum(bispec_magnitude_half[np.ix_(idx, idx)])
        band_energies[f'band_energy_{band_name}'] = band_energy

    # Bispectrum Peaks
    threshold = mean_mag + 2 * np.std(bispec_mag_flat)
    num_peaks = np.sum(bispec_mag_flat > threshold)

    # Statistical Features
    std_mag = np.std(bispec_mag_flat)
    skew_mag = skew(bispec_mag_flat)
    kurt_mag = kurtosis(bispec_mag_flat)

    # Collect all features into a dictionary
    features = {
        'mean_mag': mean_mag,
        'max_mag': max_mag,
        'sum_mag': sum_mag,
        'spectral_flatness': spectral_flatness,
        'entropy_mag': entropy_mag,
        'mean_phase': mean_phase,
        'std_phase': std_phase,
        'skew_phase': skew_phase,
        'kurt_phase': kurt_phase,
        'qpc_strength': qpc_strength,
        'qpc_freq1': qpc_freqs[0],
        'qpc_freq2': qpc_freqs[1],
        **band_energies,
        'num_peaks': num_peaks,
        'std_mag': std_mag,
        'skew_mag': skew_mag,
        'kurt_mag': kurt_mag,
    }

    return features

# Paths and parameters
output_dir = '/kaggle/working'  # Update if necessary
session = 'Interview'  # Or 'Read_Vocabulary'
participant_types = ['MDD', 'HC']
emotions = ['Positive', 'Neutral', 'Negative']

# Initialize lists to store features and labels
feature_list = []
label_list = []

# Load bispectrum data and extract features
for participant_type in participant_types:
    participant_output_dir = os.path.join(output_dir, 'results', session.lower(), participant_type)
    if not os.path.exists(participant_output_dir):
        continue

    # Assign label based on participant type
    label = 1 if participant_type == 'MDD' else 0

    # Get list of bispectrum files
    bispec_files = [f for f in os.listdir(participant_output_dir) if f.endswith('_average_bispectrum.npy')]

    for bispec_file in bispec_files:
        bispec_path = os.path.join(participant_output_dir, bispec_file)

        # Load bispectrum data
        bispec = np.load(bispec_path)

        # Extract participant ID and emotion from filename
        filename_parts = bispec_file.split('_')
        participant = filename_parts[0]
        emotion = filename_parts[2]

        # Sampling frequency
        fs = 44100  # Update if necessary

        # Extract features
        features = extract_bispectrum_features(bispec, fs)

        # Add additional information to features
        features['participant'] = participant
        features['participant_type'] = participant_type
        features['session'] = session
        features['emotion'] = emotion

        # Append features and label to lists
        feature_list.append(features)
        label_list.append(label)

# Create DataFrame from feature list
df_features = pd.DataFrame(feature_list)
df_features['label'] = label_list

# List of feature columns
feature_columns = [
    'mean_mag', 'max_mag', 'sum_mag', 'spectral_flatness', 'entropy_mag',
    'mean_phase', 'std_phase', 'skew_phase', 'kurt_phase',
    'qpc_strength', 'qpc_freq1', 'qpc_freq2',
    'band_energy_low', 'band_energy_mid', 'band_energy_high',
    'num_peaks',
    'std_mag', 'skew_mag', 'kurt_mag'
]

# Standardize the features
scaler = StandardScaler()
df_features[feature_columns] = scaler.fit_transform(df_features[feature_columns])

# Save the DataFrame with features to a CSV file
df_features.to_csv('bispectrum_features.csv', index=False)
print("Features saved to 'bispectrum_features.csv'")

# Now you have your feature matrix X and labels y
X = df_features[feature_columns].values
y = df_features['label'].values

# Proceed with machine learning modeling
# For example, train a Random Forest classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [1]:
import numpy as np
import matplotlib.pyplot as plt

def plot_bispectrum(bispec_data, nfft, fs=1):
    """
    Plot the bispectrum data using contour plot, replicating the MATLAB bispecd function.

    Parameters:
    bispec_data: 2D numpy array of bispectrum data (nfft x nfft)
    nfft: FFT length used in bispectrum computation
    fs: Sampling frequency (Hz), default is 1 as in MATLAB code
    """

    # Calculate the frequency axis (waxis)
    if nfft % 2 == 0:
        waxis = np.arange(-nfft//2, nfft//2) / nfft
    else:
        waxis = np.arange(-(nfft-1)//2, (nfft+1)//2) / nfft

    # Adjust waxis for sampling frequency
    waxis = waxis * fs

    # Shift the bispectrum data
    bispec_data_shifted = np.fft.fftshift(bispec_data)

    # Create meshgrid for plotting
    F1, F2 = np.meshgrid(waxis, waxis)

    # Plot the contour of the magnitude of the bispectrum
    plt.figure(figsize=(8, 6))
    contour_levels = 4  # Number of contour levels
    plt.contour(F1, F2, np.abs(bispec_data_shifted), levels=contour_levels)
    plt.grid(True)
    plt.title('Bispectrum estimated via the direct (FFT) method')
    plt.xlabel('f1 (Hz)')
    plt.ylabel('f2 (Hz)')

    # Set axis limits (adjust according to your data)
    fmin = -0.001 * fs
    fmax = 0.1 * fs
    plt.xlim(fmin, fmax)
    plt.ylim(fmin, fmax)

    # Overlay lines
    # Line f1 = f2
    x_line = np.linspace(max(0, fmin), fmax, 500)
    plt.plot(x_line, x_line, 'r', label='f1 = f2')

    # Line f1 + f2 = fs / 2
    f1_line = np.linspace(max(0, fmin), fmax, 500)
    f2_line = (fs / 2) - f1_line
    # Only plot where f2_line is within the axis limits
    valid_idx = (f2_line >= fmin) & (f2_line <= fmax)
    plt.plot(f1_line[valid_idx], f2_line[valid_idx], 'r', label='f1 + f2 = fs / 2')

    # Line f2 = 0
    plt.plot([fmin, fmax], [0, 0], 'r', label='f2 = 0')

    # Add legend
    plt.legend()

    plt.show()


In [4]:
# unzip data
!unzip /content/bispectrum_data.zip -d /content/Bispectorum_data

Archive:  /content/bispectrum_data.zip
warning [/content/bispectrum_data.zip]:  2097152 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  2097152
  (attempting to re-compensate)
   creating: /content/Bispectorum_data/interview/HC/
error: invalid zip file with overlapped components (possible zip bomb)


In [ ]:
import numpy as np

# Path to a bispectrum data file
bispec_file_path = '/path/to/your/bispectrum_data.npy'

# Load the bispectrum data
bispec_data = np.load(bispec_file_path)
